In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,450 kB]
Get:14 http://a

In [2]:
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver
!pip install chromedriver-autoinstaller

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [3]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.26.0


In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [5]:
chrome_path = "/usr/lib/chromium-browser/chromedriver"

In [6]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/130/chromedriver'

In [33]:
driver = webdriver.Chrome(options=chrome_options)

In [8]:
from datetime import datetime, timedelta

# 오늘 날짜 계산
today = datetime.now().date()
ninety_days_ago = today - timedelta(days=90)

In [9]:
import re

In [36]:
# request_chart_type=['회사채 신용 스프레드']

url = "https://snapshot.bok.or.kr/dashboard/E1"
driver.get(url)

time.sleep(3)

path_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#highcharts-zqgdtpd-0 > svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-0.highcharts-line-series.highcharts-color-0"))
    )

d_value = path_element.get_attribute("d")
data = []



for point in d_value[:90]:
  webdriver.ActionChains(driver).move_to_element(point).perform()
  time.sleep(0.5)
  tooltip_element = driver.find_element(By.CSS_SELECTOR, "text[data-z-index='1']")

  # 요소의 내부 HTML 가져오기
  tooltip_html = tooltip_element.get_attribute('innerHTML')

  # 날짜 정보 추출 (정규표현식 사용)
  date_pattern = r'<tspan style="font-size: 10px;">(.*?)</tspan>'
  date_match = re.search(date_pattern, tooltip_html)
  date = date_match.group(1) if date_match else 'No Date Found'

  # 채권 정보 추출 (정규표현식 사용)
  bond_pattern = r'([가-힣 ]+\(\d+년\)): <tspan style="font-weight: bold;">(\d+\.\d+)</tspan>'
  bond_matches = re.findall(bond_pattern, tooltip_html)

  data.append({'Date': date, 'Value': bond_matches})

  # 날짜순으로 정렬
  data.sort(key=lambda x: x['Date'])


# 데이터프레임 생성
df = pd.DataFrame(data)

display(df)
# CSV 파일로 저장
# df.to_csv('company_bond_rates.csv', index=False)

driver.quit()

TimeoutException: Message: 
Stacktrace:
#0 0x5ce0e52bc13a <unknown>
#1 0x5ce0e4dd7460 <unknown>
#2 0x5ce0e4e23f96 <unknown>
#3 0x5ce0e4e24221 <unknown>
#4 0x5ce0e4e695d4 <unknown>
#5 0x5ce0e4e4803d <unknown>
#6 0x5ce0e4e66af6 <unknown>
#7 0x5ce0e4e47db3 <unknown>
#8 0x5ce0e4e16c10 <unknown>
#9 0x5ce0e4e17bee <unknown>
#10 0x5ce0e52883cb <unknown>
#11 0x5ce0e528c368 <unknown>
#12 0x5ce0e5275eec <unknown>
#13 0x5ce0e528cee7 <unknown>
#14 0x5ce0e525b13f <unknown>
#15 0x5ce0e52aa858 <unknown>
#16 0x5ce0e52aaa20 <unknown>
#17 0x5ce0e52bafb6 <unknown>
#18 0x7af713c74ac3 <unknown>


In [19]:
data

[]

In [26]:
points

[]